Toronto's Open Data portal has some neat stuff.

In this notebook, we'll explore the data and analyze and aggregate it. We'll work with the 2017 Speed Data

In the next notebook, we'll put the results on maps.

Data sources:
Download "bluetooth-routes-wgs84" and "travel-time-2017" from here:
https://open.toronto.ca/dataset/travel-times-bluetooth/

Put them somewhere like "documents/data"

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
import os
import pylab as P

In [2]:
data_path = '/Users/jeffsalvail/documents/data'
os.chdir(data_path)

The first file, the shapefile (extension .shp) is essentially the digital map. Segments identified by unique identifiers (called 'resultId'), and their geographic coordinates (under the heading 'geometry'). We'll map this on the next notebook.

In [3]:
df_routes = gpd.read_file('shapefile/bluetooth_routes_wgs84.shp')
df_routes[:10]

,resultId,normalDriv,length_m,geometry
0,B_A,114,3179,LINESTRING (-79.49349487364718 43.624733955612...
1,A_B,114,3181,LINESTRING (-79.53129990829397 43.616722988484...
2,B_B1,164,4556,LINESTRING (-79.49340465372515 43.624530277107...
3,B1_B,164,4565,LINESTRING (-79.44330649895257 43.636703605749...
4,B1_C,66,1848,LINESTRING (-79.44341260415925 43.636597385124...
5,C_B1,66,1833,LINESTRING (-79.42260900075053 43.634639293488...
6,C_D,91,2537,LINESTRING (-79.42253713127013 43.634528379083...
7,D_C,91,2541,LINESTRING (-79.3920304484344 43.6387985801261...
8,D_E,104,2898,LINESTRING (-79.39199170486174 43.638685534618...
9,E_D,104,2894,LINESTRING (-79.3591643218594 43.6485896612959...


The next piece we need is the speed data itself.

The speed data (in the file 'bt_2017.csv') reports the 'median travel time' according to bluetooth detectors for each segment of road. These segments are the same 'resultId' as in the shapefile! That will be "key" (pun intended - 'resultId' is sometimes called the key).

The speed observations are for every 5 minutes in January, for every segment. It's a big file! Yay computers.

The next cell loads the in the .csv file and cleans up the date-time column to make it more useful. It also removes rows where the travel-time was indicated as 0s. That's nonsense. Setting these rows to 'NaN' (Not a Number) is the easiest way to make the averaging ignore these rows. There are other ways but I like this way.

In [4]:
df_bt = pd.read_csv('bt_2017.csv')

def trim_last_N(row):
    return row[:-3]

df_bt['updated'] = df_bt['updated'].apply(trim_last_N)
df_bt['updated'] = pd.to_datetime(df_bt['updated'])

zero_time_indices = df_bt[df_bt['timeInSeconds']==0].index #remove travel-time = 0, nonsense
df_bt.loc[zero_time_indices]=np.nan

df_bt[:10]

,resultId,timeInSeconds,count,updated
0,J_I,56.0,16.0,2017-01-01 00:05:00
1,J_I,60.0,9.0,2017-01-01 00:10:00
2,J_I,60.0,8.0,2017-01-01 00:15:00
3,J_I,60.0,8.0,2017-01-01 00:20:00
4,J_I,55.0,17.0,2017-01-01 00:25:00
5,J_I,53.0,15.0,2017-01-01 00:30:00
6,J_I,58.0,13.0,2017-01-01 00:35:00
7,J_I,55.0,18.0,2017-01-01 00:40:00
8,J_I,58.0,22.0,2017-01-01 00:45:00
9,J_I,54.0,19.0,2017-01-01 00:50:00


Now we'll start doing things!

We want to compute the 'speed' vehicles are travelling. Speed is distance/time. We have time, we need distance. That is in the shapefile under the column 'length_m'. All we need to do is convert the length_m to km, and the travel time in seconds (timeInSeconds in the data table) to hours. Then we can divide them to get the kilometres per hour.

To do that, we need to "join" the tables. This is where the 'resultId' comes in handy. We'll join the tables where rows have the same 'resultId', then do the math and make a new column for speed in kph.

In [5]:
df_joined = df_bt.set_index(['resultId',]).join(df_routes.set_index('resultId'),how='right').drop(['normalDriv','geometry'],axis=1)
df_joined['median_speed_kph'] = (df_joined['length_m']/1000.)/(df_joined['timeInSeconds']/3600.)
df_joined.astype({'median_speed_kph':np.float64})
df_joined[:10]

,timeInSeconds,count,updated,length_m,median_speed_kph
resultId,,,,,
AC1_AC2,369.0,4.0,2017-01-01 00:05:00,1559,15.209756
AC1_AC2,441.0,5.0,2017-01-01 00:10:00,1559,12.726531
AC1_AC2,445.0,3.0,2017-01-01 00:15:00,1559,12.612135
AC1_AC2,498.0,3.0,2017-01-01 00:20:00,1559,11.269880
AC1_AC2,331.0,3.0,2017-01-01 00:25:00,1559,16.955891
AC1_AC2,398.0,7.0,2017-01-01 00:30:00,1559,14.101508
AC1_AC2,410.0,3.0,2017-01-01 00:35:00,1559,13.688780
AC1_AC2,380.0,6.0,2017-01-01 00:40:00,1559,14.769474
AC1_AC2,373.0,3.0,2017-01-01 00:50:00,1559,15.046649


The table df_joined has the median speed for each segment, every 5 minutes in January.

That's a good start. One thing people do in the traffic world is compare the median speed to the "free-flow" speed (FFS). That's usually defined as the overnight speed. When there are fewer cars on the road, we can assume it's less congested and vehicles travel at the "ideal" speed for that segment of road.

Let's compute the FFS for each link using a pivot table. We want only the records between 9pm and 6am (overnight). We'll get the average overnight speed for each link.

In [6]:
df_overnight = df_joined.reset_index().set_index('updated').between_time('21:00','06:00')
df_ffs = pd.pivot_table(df_overnight,
                       index='resultId',
                       values='median_speed_kph',
                       aggfunc=np.nanmean,
                       ).rename(columns={"median_speed_kph": "FFS",})
df_ffs[:10]

,FFS
resultId,
AC1_AC2,22.200678
AC1_AD3,21.272678
AC2_AC1,22.780166
AD1_AD2,33.535992
AD2_AD1,33.676336
AD2_AD3,25.353012
AD3_AC1,23.327818
AD3_AD2,25.935554
AD3_AD4,23.449969


This little table is the FFS for each link. We'll join that to the original table, so we can divide every speed record by the FFS.

The ratio FFS/Travel Speed is a dimensionless parameter called travel-time index (TTI).

In [7]:
df_joined = df_joined.join(df_ffs)
df_joined['TTI']=df_joined['FFS']/df_joined['median_speed_kph']
df_joined[:10]

,timeInSeconds,count,updated,length_m,median_speed_kph,FFS,TTI
resultId,,,,,,,
AC1_AC2,369.0,4.0,2017-01-01 00:05:00,1559,15.209756,22.200678,1.459634
AC1_AC2,441.0,5.0,2017-01-01 00:10:00,1559,12.726531,22.200678,1.744441
AC1_AC2,445.0,3.0,2017-01-01 00:15:00,1559,12.612135,22.200678,1.760263
AC1_AC2,498.0,3.0,2017-01-01 00:20:00,1559,11.269880,22.200678,1.969913
AC1_AC2,331.0,3.0,2017-01-01 00:25:00,1559,16.955891,22.200678,1.309319
AC1_AC2,398.0,7.0,2017-01-01 00:30:00,1559,14.101508,22.200678,1.574348
AC1_AC2,410.0,3.0,2017-01-01 00:35:00,1559,13.688780,22.200678,1.621816
AC1_AC2,380.0,6.0,2017-01-01 00:40:00,1559,14.769474,22.200678,1.503146
AC1_AC2,373.0,3.0,2017-01-01 00:50:00,1559,15.046649,22.200678,1.475457


Great! Now we have the TTI for each link, for each 5 minute period of time in January...

Now what?

Notebook 2 will aggregate these and put them on a map. The last thing we should do before logging off is save these rsults to a table.

Which columns do we want to keep? The index ('resultId') is most important. The 'updated' (date-time) is super important as well. Let's keep the counts, speed, FFS and TTI also and use all those in the maps.

In [11]:
df_out = df_joined[['updated','median_speed_kph','FFS','TTI','count']]
df_out.to_csv('toronto_speed_data.csv')
df_out[:10]

,updated,median_speed_kph,FFS,TTI,count
resultId,,,,,
AC1_AC2,2017-01-01 00:05:00,15.209756,22.200678,1.459634,4.0
AC1_AC2,2017-01-01 00:10:00,12.726531,22.200678,1.744441,5.0
AC1_AC2,2017-01-01 00:15:00,12.612135,22.200678,1.760263,3.0
AC1_AC2,2017-01-01 00:20:00,11.269880,22.200678,1.969913,3.0
AC1_AC2,2017-01-01 00:25:00,16.955891,22.200678,1.309319,3.0
AC1_AC2,2017-01-01 00:30:00,14.101508,22.200678,1.574348,7.0
AC1_AC2,2017-01-01 00:35:00,13.688780,22.200678,1.621816,3.0
AC1_AC2,2017-01-01 00:40:00,14.769474,22.200678,1.503146,6.0
AC1_AC2,2017-01-01 00:50:00,15.046649,22.200678,1.475457,3.0
